# GONZAG cloud example

In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
from gonzag.config import *
from gonzag.io import *
from gonzag.utils import *
from gonzag.mod2sat import *
from gonzag.bilin_mapping import *
from gonzag.spectralysis import *


## Data

In [3]:
import xarray as xr
osn_url = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/swot_adac/eNATL60_surface_region_1'
dsmod = xr.open_zarr(osn_url, consolidated=True)

In [4]:
dsmod

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 611, y: 763)
Coordinates:
    depth          (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lat            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables:
    fmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosstsst       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>

In [5]:
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
l_griddist = False

In [6]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat["al"].to_dask()

In [7]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185024 ... 2...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    creator_name:              CMEMS - Sea Level Thematic Assembly Center
    creator_url:               http://marine.copernicus.eu
    history:                   2019-02-20T19:39:52Z: Creation
    institution:               CLS, CNES
    keywords:                  Oceans > Ocean Topography > Sea Surface Height
    keywords_vocabulary:       NetCDF COARDS Climate and Forecast Standard Names
    license:                   http://marine.copernicus.eu/web/27-service-com...
    platform:                  Altika
    processing_level:          L3
    product_version:           2019
    project:                   COPERNICUS MARINE ENVIRONMENT MONITORING SERVI...
    references:                http://marine.copernicus.eu
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [8]:
name_ssh_sat='sla_unfiltered'

## Time overlap between model and satellite ?
Assume that model is less than 1 year, satellite has at least one whole year, we take the overlapping period without considering which year is this

In [9]:
    (date1,date2), (Nts,Ntm) = GetTimeOverlap( dssat , dsmod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-02-01 -- 2014-04-30


## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [10]:
  
    ModelGrid = ModGrid( dsmod, date1 , date2 , dsmod, name_lsm_mod, distorded_grid=False )



 *** what we use to define model land-sea mask:
    => "tmask" in dataset 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (763, 611)
     * horizontal resolution:  0.016657295  degrees or  1.8507920147292316  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  281.99 292.17
     * lat_min, lat_max =  30.01 40.01
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2136
       ==> time record dates: 2014-02-01 to 2014-04-30, included



In [11]:
SatelliteTrack = SatTrack( dssat, date1, date2, name_ssh_sat, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  10236
       ==> time record indices: 14883510 to 19033581, included

       separated in 86 tracks


In [12]:
def process_one_track(track):
    Solution0 = Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track )
    return Solution0


In [13]:
result = process_one_track(27)

 *** "found" distance criterion when searching for nearest point on model grid is  1.3880940110469238  km

 *** Will use zoom boxes of width of 271 points for 1st attempts of nearest-point location...


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3880940110469238 135 )
      +++ Treated point: 1/173 
          ==> Sat. coordinates:     40.002 286.099
          ==> Model nearest point:  40.007 292.013  ( -1 -1 )
      +++ Treated point: 2/173 
          ==> Sat. coordinates:     39.944 286.08
          ==> Model nearest point:  40.007 292.013  ( -1 -1 )
      +++ Treated point: 3/173 
          ==> Sat. coordinates:     39.887 286.062
          ==> Model nearest point:  40.007 292.013  ( -1 -1 )
      +++ Treated point: 4/173 
          ==> Sat. coordinates:     39.829 286.043
          ==> Model nearest point:  39.83 286.04  ( 758 251 )
      +++ Treated point: 5/173 
          ==> Sat. coordinates:     39.772 286.024
          ==> Model nearest poi

In [14]:
result.ssh_sat

masked_array(data=[ 0.162,  0.166,  0.138,  0.151,  0.171,  0.176,  0.156,
                    0.134,  0.126,  0.147,  0.105,  0.114,  0.119,  0.138,
                    0.113,  0.117,  0.118,  0.121,  0.135,  0.132,  0.116,
                    0.113,  0.12 ,  0.126,  0.121,  0.121,  0.103,  0.103,
                    0.126,  0.086,  0.078,  0.081,  0.096,  0.086,  0.102,
                    0.116,  0.136,  0.12 ,  0.11 ,  0.142,  0.181,  0.177,
                    0.149,  0.138,  0.124,  0.132,  0.145,  0.108,  0.091,
                    0.115,  0.131,  0.099,  0.101,  0.086,  0.096,  0.103,
                    0.11 ,  0.115,  0.069,  0.085,  0.093,  0.106,  0.107,
                    0.124,  0.119,  0.087,  0.089,  0.116,  0.106,  0.112,
                    0.107,  0.097,  0.087,  0.094,  0.102,  0.081,  0.092,
                    0.098,  0.069,  0.069,  0.05 ,  0.069,  0.068,  0.062,
                    0.071,  0.056,  0.076,  0.074,  0.068,  0.088,  0.072,
                    0.077

In [15]:
        one_track=27
        jt1=SatelliteTrack.index_tracks[one_track][0]
        jt2=SatelliteTrack.index_tracks[one_track][1]


In [16]:
print(jt1,jt2)

6930 7103


In [17]:
vssh_s = GetSatSSH( SatelliteTrack.file, name_ssh_sat,  kt1=jt1, kt2=jt2-1)

In [18]:
vssh_s

<xarray.DataArray 'sla_unfiltered' (time: 173)>
array([ 0.162,  0.166,  0.138,  0.151,  0.171,  0.176,  0.156,  0.134,
        0.126,  0.147,  0.105,  0.114,  0.119,  0.138,  0.113,  0.117,
        0.118,  0.121,  0.135,  0.132,  0.116,  0.113,  0.12 ,  0.126,
        0.121,  0.121,  0.103,  0.103,  0.126,  0.086,  0.078,  0.081,
        0.096,  0.086,  0.102,  0.116,  0.136,  0.12 ,  0.11 ,  0.142,
        0.181,  0.177,  0.149,  0.138,  0.124,  0.132,  0.145,  0.108,
        0.091,  0.115,  0.131,  0.099,  0.101,  0.086,  0.096,  0.103,
        0.11 ,  0.115,  0.069,  0.085,  0.093,  0.106,  0.107,  0.124,
        0.119,  0.087,  0.089,  0.116,  0.106,  0.112,  0.107,  0.097,
        0.087,  0.094,  0.102,  0.081,  0.092,  0.098,  0.069,  0.069,
        0.05 ,  0.069,  0.068,  0.062,  0.071,  0.056,  0.076,  0.074,
        0.068,  0.088,  0.072,  0.077,  0.095,  0.083,  0.058,  0.063,
        0.091,  0.083,  0.041,  0.021,  0.029,  0.018,  0.005,  0.002,
       -0.003, -0.002, -0.017, -0.031,  0.181,  0.111,  0.152,  0.186,
        0.205,  0.211,  0.227,  0.256,  0.289,  0.329,  0.385,  0.382,
        0.357,  0.379,  0.391,  0.388,  0.368,  0.36 ,  0.353,  0.331,
        0.322,  0.327,  0.336,  0.33 ,  0.316,  0.341,  0.339,  0.341,
        0.326,  0.315,  0.303,  0.302,  0.307,  0.297,  0.302,  0.303,
        0.307,  0.308,  0.301,  0.298,  0.294,  0.303,  0.31 ,  0.307,
        0.298,  0.308,  0.305,  0.294,  0.284,  0.309,  0.313,  0.308,
        0.301,  0.296,  0.303,  0.301,  0.299,  0.292,  0.292,  0.296,
        0.293,  0.287,  0.286,  0.28 ,  0.296])
Coordinates:
    latitude   (time) float64 3.588 3.529 3.471 3.413 ... -12.41 -12.46 -12.52
    longitude  (time) float64 143.2 143.2 143.2 143.2 ... 139.7 139.7 139.6
  * time       (time) datetime64[ns] 2013-03-14T08:34:28.619737088 ... 2013-0...
Attributes:
    comment:        The sea level anomaly is the sea surface height above mea...
    long_name:      Sea level anomaly not-filtered not-subsampled with dac, o...
    standard_name:  sea_surface_height_above_sea_level
    units:          m